# Lectura 37: LazyFrame - Manipulación y selección

Debemos tener en cuenta que estas operaciones funcionan de la misma forma a como lo hacen con los DataFrame con algunas excepciones. Como ya vimos una gran variedad de estas funciones en la sección anterior del curso en esta ocasión solo mostraremos algunos ejemplos.

In [1]:
import polars as pl

vuelos_lf = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
)

## `botton_k`

Retorna los k elementos más pequeños del DataFrame. Le podemos indicar la cantidad de elementos que deseamos mostrar y por cual columna ordenar para obtener el resultado.

In [2]:
vuelos_lf.bottom_k(20, by='DEPARTURE_TIME', descending=True, nulls_last=True).collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,9,8,2,"""WN""",2830,"""N649SW""","""MSY""","""HOU""",2135,2400,145,6,6,70,53,45,302,51,2,2245,53,128,0,0,null,0,0,0,128,0
2015,2,17,2,"""US""",2070,"""N755US""","""CLT""","""CLE""",2220,2400,100,12,12,92,81,62,430,114,7,2352,121,89,0,0,null,0,0,3,86,0
2015,10,21,3,"""B6""",8,"""N649JB""","""12889""","""11697""",2359,2400,1,11,11,278,267,252,2173,723,4,737,727,-10,0,0,null,null,null,null,null,null
2015,7,8,3,"""NK""",640,"""N515NK""","""LAS""","""BOS""",2310,2400,50,13,13,313,293,270,2381,743,10,723,753,30,0,0,null,1,0,0,29,0
2015,3,29,7,"""US""",529,"""N912UY""","""PHX""","""ORD""",2359,2400,1,19,19,202,190,164,1440,503,7,521,510,-11,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,2,15,7,"""AA""",2400,"""N5DJAA""","""LAX""","""DFW""",5,2400,-5,19,19,168,171,148,1235,447,4,453,451,-2,0,0,null,null,null,null,null,null
2015,2,17,2,"""US""",1717,"""N114UW""","""CLT""","""FLL""",2210,2400,110,14,14,113,120,101,631,155,5,3,200,117,0,0,null,7,0,0,110,0
2015,5,17,7,"""WN""",508,"""N7741C""","""MDW""","""ORF""",2150,2400,130,5,5,115,99,89,704,234,5,45,239,114,0,0,null,0,0,0,114,0


## `drop` 

Elimina la(s) columna(s) seleccionada(s).

In [3]:
vuelos_lf.drop('YEAR', 'MONTH', 'DAY').collect()

DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,null,null,null,null,null,null
4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,null,null,null,null,null,null
4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,null,null,null,null,null,null
4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
4,"""B6""",688,"""N657JB""","""LAX""","""BOS""",2359,2355,-4,22,17,320,298,272,2611,749,4,819,753,-26,0,0,null,null,null,null,null,null
4,"""B6""",745,"""N828JB""","""JFK""","""PSE""",2359,2355,-4,17,12,227,215,195,1617,427,3,446,430,-16,0,0,null,null,null,null,null,null
4,"""B6""",1503,"""N913JB""","""JFK""","""SJU""",2359,2350,-9,17,7,221,222,197,1598,424,8,440,432,-8,0,0,null,null,null,null,null,null


## Filtrar un DataFrame

In [4]:
from polars import col

(
    vuelos_lf
    .filter((col('MONTH') > 6) & (col('DEPARTURE_DELAY') < 0))
    .select(
        col('MONTH'),
        col('DEPARTURE_DELAY')
    ).collect()
)

MONTH,DEPARTURE_DELAY
i32,i32
7,-5
7,-5
7,-5
7,-7
7,-2
…,…
12,-1
12,-4
12,-4


## `with_context`

Esta función agrega un contexto externo al gráfico de cálculo. Esto permite que las expresiones también accedan a columnas de DataFrames que no forman parte de este.

In [5]:
tarjetas_lz = pl.LazyFrame({'tarjeta': ['bronce', 'plata', 'oro']})

descuentos_lz = pl.LazyFrame({'descuento': [10, 20, 30]})

tarjetas_lz.with_context(descuentos_lz).select(
    col('tarjeta'),
    col('descuento')
).collect()

tarjeta,descuento
str,i64
"""bronce""",10
"""plata""",20
"""oro""",30


## `slice`

Con esta función podemos obtener una porción o lo que en inglés se conoce como un slice de un DataFrame.

In [6]:
vuelos_lf.slice(1,10).collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,1,4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""US""",840,"""N171US""","""SFO""","""CLT""",20,18,-2,16,34,286,293,266,2296,800,11,806,811,5,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",258,"""N3HYAA""","""LAX""","""MIA""",20,15,-5,15,30,285,281,258,2342,748,8,805,756,-9,0,0,null,null,null,null,null,null
2015,1,1,4,"""AS""",135,"""N527AS""","""SEA""","""ANC""",25,24,-1,11,35,235,215,199,1448,254,5,320,259,-21,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",806,"""N3730B""","""SFO""","""MSP""",25,20,-5,18,38,217,230,206,1589,604,6,602,610,8,0,0,null,null,null,null,null,null
2015,1,1,4,"""NK""",612,"""N635NK""","""LAS""","""MSP""",25,19,-6,11,30,181,170,154,1299,504,5,526,509,-17,0,0,null,null,null,null,null,null
2015,1,1,4,"""US""",2013,"""N584UW""","""LAX""","""CLT""",30,44,14,13,57,273,249,228,2125,745,8,803,753,-10,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",1112,"""N3LAAA""","""SFO""","""DFW""",30,19,-11,17,36,195,193,173,1464,529,3,545,532,-13,0,0,null,null,null,null,null,null
2015,1,1,4,"""DL""",1173,"""N826DN""","""LAS""","""ATL""",30,33,3,12,45,221,203,186,1747,651,5,711,656,-15,0,0,null,null,null,null,null,null


## `sort`

Esta función ordena el LazyFrame por la(s) columna(s) proporcionadas.

In [7]:
vuelos_lf.sort('DEPARTURE_DELAY', descending=True, nulls_last=True).collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,23,5,"""AA""",1322,"""N598AA""","""BHM""","""DFW""",700,1608,1988,13,1621,134,117,99,597,1800,5,914,1805,1971,0,0,null,0,0,1971,0,0
2015,8,1,6,"""AA""",96,"""N479AA""","""RIC""","""DFW""",709,1427,1878,28,1455,185,205,156,1158,1631,21,914,1652,1898,0,0,null,20,0,1878,0,0
2015,9,13,7,"""AA""",1063,"""N3CAAA""","""SAN""","""DFW""",700,1050,1670,26,1116,179,174,142,1171,1538,6,1159,1544,1665,0,0,null,0,0,1665,0,0
2015,12,31,4,"""AA""",2214,"""N4XKAA""","""ABQ""","""DFW""",1041,1410,1649,12,1422,104,91,69,569,1631,10,1325,1641,1636,0,0,null,0,0,1636,0,0
2015,11,27,5,"""AA""",2559,"""N489AA""","""DTW""","""ORD""",1027,1338,1631,14,1352,82,89,56,235,1348,19,1049,1407,1638,0,0,null,7,0,1631,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,1,1,4,"""OO""",5460,"""N583SW""","""RDD""","""SFO""",530,null,null,null,null,90,null,null,199,null,null,700,null,null,0,1,"""A""",null,null,null,null,null
2015,1,1,4,"""MQ""",2859,"""N660MQ""","""SGF""","""DFW""",525,null,null,null,null,95,null,null,364,null,null,700,null,null,0,1,"""B""",null,null,null,null,null
2015,1,1,4,"""OO""",5254,"""N746SK""","""MAF""","""IAH""",510,null,null,null,null,87,null,null,429,null,null,637,null,null,0,1,"""B""",null,null,null,null,null


## `gather_every`

Toma cada enésima fila del LazyFrame y regresa un nuevo LazyFrame. El primer parámetro estable cada cuantas filas queremos tomar la siguiente fila y el parámetro offset establece desde que posición queremos iniciar a tomar las filas.

In [8]:
vuelos_lf.gather_every(5, 100).collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
2015,1,1,4,"""EV""",5498,"""N132EV""","""ORD""","""DTW""",535,550,15,18,608,85,67,41,235,749,8,800,757,-3,0,0,null,null,null,null,null,null
2015,1,1,4,"""B6""",1012,"""N375JB""","""ORD""","""BOS""",540,529,-11,16,545,136,114,95,867,820,3,856,823,-33,0,0,null,null,null,null,null,null
2015,1,1,4,"""EV""",4160,"""N11150""","""JAX""","""EWR""",540,531,-9,9,540,137,132,109,820,729,14,757,743,-14,0,0,null,null,null,null,null,null
2015,1,1,4,"""OO""",4833,"""N554CA""","""MKE""","""DTW""",545,543,-2,10,553,74,55,36,237,729,9,759,738,-21,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",2299,"""N3LLAA""","""JFK""","""MIA""",545,640,55,17,657,185,199,152,1089,929,30,850,959,69,0,0,null,14,0,55,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,12,31,4,"""AS""",471,"""N570AS""","""LAX""","""SEA""",2355,2353,-2,17,10,169,152,132,954,222,3,244,225,-19,0,0,null,null,null,null,null,null
2015,12,31,4,"""F9""",682,"""N701FR""","""DEN""","""MCO""",2359,2354,-5,12,6,206,190,170,1546,456,8,525,504,-21,0,0,null,null,null,null,null,null
2015,12,31,4,"""UA""",671,"""N33209""","""PDX""","""IAH""",2359,2352,-7,10,2,245,233,214,1825,536,9,604,545,-19,0,0,null,null,null,null,null,null


## `shift`

Esta función cambia los valores según el número dado de índices. En la práctica lo podemos ver como un desplazamiento de las filas del LazyFrame.

In [9]:
vuelos_lf.shift(3).collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
i32,i32,i32,i32,str,i32,str,str,str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,str,i32,i32,i32,i32,i32
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2015,1,1,4,"""AS""",98,"""N407AS""","""ANC""","""SEA""",5,2354,-11,21,15,205,194,169,1448,404,4,430,408,-22,0,0,null,null,null,null,null,null
2015,1,1,4,"""AA""",2336,"""N3KUAA""","""LAX""","""PBI""",10,2,-8,12,14,280,279,263,2330,737,4,750,741,-9,0,0,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2015,12,31,4,"""B6""",802,"""N589JB""","""SLC""","""MCO""",2359,15,16,14,29,249,250,211,1931,600,25,608,625,17,0,0,null,1,0,16,0,0
2015,12,31,4,"""B6""",98,"""N607JB""","""DEN""","""JFK""",2359,6,7,13,19,211,193,173,1626,512,7,530,519,-11,0,0,null,null,null,null,null,null
2015,12,31,4,"""B6""",66,"""N655JB""","""ABQ""","""JFK""",2359,15,16,9,24,227,214,190,1826,534,15,546,549,3,0,0,null,null,null,null,null,null


In [10]:
vuelos_lf.shift(3, fill_value='prueba').collect()

YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba""","""prueba"""
"""2015""","""1""","""1""","""4""","""AS""","""98""","""N407AS""","""ANC""","""SEA""","""5""","""2354""","""-11""","""21""","""15""","""205""","""194""","""169""","""1448""","""404""","""4""","""430""","""408""","""-22""","""0""","""0""",null,null,null,null,null,null
"""2015""","""1""","""1""","""4""","""AA""","""2336""","""N3KUAA""","""LAX""","""PBI""","""10""","""2""","""-8""","""12""","""14""","""280""","""279""","""263""","""2330""","""737""","""4""","""750""","""741""","""-9""","""0""","""0""",null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2015""","""12""","""31""","""4""","""B6""","""802""","""N589JB""","""SLC""","""MCO""","""2359""","""15""","""16""","""14""","""29""","""249""","""250""","""211""","""1931""","""600""","""25""","""608""","""625""","""17""","""0""","""0""",null,"""1""","""0""","""16""","""0""","""0"""
"""2015""","""12""","""31""","""4""","""B6""","""98""","""N607JB""","""DEN""","""JFK""","""2359""","""6""","""7""","""13""","""19""","""211""","""193""","""173""","""1626""","""512""","""7""","""530""","""519""","""-11""","""0""","""0""",null,null,null,null,null,null
"""2015""","""12""","""31""","""4""","""B6""","""66""","""N655JB""","""ABQ""","""JFK""","""2359""","""15""","""16""","""9""","""24""","""227""","""214""","""190""","""1826""","""534""","""15""","""546""","""549""","""3""","""0""","""0""",null,null,null,null,null,null


## `inspect`

Esta función inspecciona un nodo en el gráfico de cálculo. Con ella podemos imprimir el valor que evalúa este nodo en el gráfico de cálculo.

In [11]:
query_inspect = (
    vuelos_lf.with_columns(AIR_TIME_HRS=(col('AIR_TIME') / 60))
    .inspect()
    .select(
        col('AIRLINE'),
        col('AIR_TIME_HRS')
    )
)

In [12]:
query_inspect.collect()

shape: (5_819_079, 3)
┌─────────┬──────────┬──────────────┐
│ AIRLINE ┆ AIR_TIME ┆ AIR_TIME_HRS │
│ ---     ┆ ---      ┆ ---          │
│ str     ┆ i32      ┆ f64          │
╞═════════╪══════════╪══════════════╡
│ AS      ┆ 169      ┆ 2.816667     │
│ AA      ┆ 263      ┆ 4.383333     │
│ US      ┆ 266      ┆ 4.433333     │
│ AA      ┆ 258      ┆ 4.3          │
│ AS      ┆ 199      ┆ 3.316667     │
│ …       ┆ …        ┆ …            │
│ B6      ┆ 272      ┆ 4.533333     │
│ B6      ┆ 195      ┆ 3.25         │
│ B6      ┆ 197      ┆ 3.283333     │
│ B6      ┆ 144      ┆ 2.4          │
│ B6      ┆ 189      ┆ 3.15         │
└─────────┴──────────┴──────────────┘


AIRLINE,AIR_TIME_HRS
str,f64
"""AS""",2.816667
"""AA""",4.383333
"""US""",4.433333
"""AA""",4.3
"""AS""",3.316667
…,…
"""B6""",4.533333
"""B6""",3.25
"""B6""",3.283333


In [14]:
print(query_inspect.explain())

 SELECT [col("AIRLINE"), col("AIR_TIME_HRS")] FROM
  python dataframe udf
     WITH_COLUMNS:
     [[(col("AIR_TIME")) / (60)].alias("AIR_TIME_HRS")]

        Parquet SCAN ./data/vuelos/vuelos.parquet
        PROJECT 2/31 COLUMNS


Hasta aquí llegaremos en este recorrido de las funciones que nos permiten manipular y seleccionar elementos de un LazyFrame. En de que ustedes necesiten ver la lista competa de todas las funciones que están disponibles pueden dirigirse a la documentación de Polars.

[LazyFrame - Manipulación y selección](https://docs.pola.rs/py-polars/html/reference/lazyframe/modify_select.html)